In [1]:
!pip install pyspark==3.2.1
# echo $PATH 로 /spark/bin 이 정상적으로 로드되서, pyspark 설치하지 않은 상태에서 명령창에서 pyspark로 실행 가능해야 한다.  
# 여기서 설치하는 pyspark는 spark 연동 위한 api 용. 
# pyspark 명령창 실행은 주피터 실행 전에 환경 점검하고, 이를 그대로 넘겨 받는다.  
# spark session은 결국 외부 명령창에서 pyspark를 실행하는 형태를 연결해 줄 뿐.  
# pyspark 설치 전에 !which pyspark로 확인하면 : /spark/bin 이 출력되고, 
# 설치 후에 확인하면 /usr/local/bin 이 출력된다.  

    100% |################################| 281.4MB 4.3kB/s eta 0:00:01   24% |#######                         | 70.0MB 46.9MB/s eta 0:00:05
    100% |################################| 204kB 5.9MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyspark ... done
  Stored in directory: /root/.cache/pip/wheels/23/35/61/f720c76ff25c680383cd010cd524fa66c91c9b62a4b0aebe9f
Successfully built pyspark


In [3]:
!echo $PATH

/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin:/hadoop/bin:/hadoop/sbin:/spark/bin


In [4]:
!pip install findspark

In [5]:
# import libraries 
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
import findspark
import time 

In [6]:
# 전역변수 설정 
# local mode 
SPARK_LOCAL_MASTER = "local[3]"
# client mode 
SPARK_CLUSTER_MASTER = "yarn" #"spark://35.202.237.174:7077" 
SPARK_APP_NAME = "DATA-Preparation"
HOST_NAME = "spark-master" #socket.gethostname()


# Define path
DATA_ROOT = f'{os.getcwd()}/data'
DATA_PATH = f'{DATA_ROOT}'

In [7]:
print(DATA_PATH)
print(HOST_NAME)

/notebooks/data
spark-master


In [12]:
os.environ['SPARK_HOME'] = "/spark"

In [13]:
findspark.init()
print(os.getenv('SPARK_HOME'))

/spark


In [14]:
print(f"JAVA_HOME : {os.getenv('JAVA_HOME')}")
print(f"PATH : {os.getenv('PATH')}")
os.getcwd()

JAVA_HOME : None
PATH : /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin:/hadoop/bin:/hadoop/sbin:/spark/bin


'/notebooks'

In [10]:
# Spark session 생성 메서드 
def init_remote_session():
    #SPARK_CLUSTER_MASTER = "spark://34.64.108.172:7077" 
    spark = SparkSession.builder.master(SPARK_CLUSTER_MASTER).appName(SPARK_APP_NAME).getOrCreate()
    return spark
    
# local mode 실행 시 메모리를 확장해야 하는 경우 있어서, conf에 memory 변경 추가  
def init_local_session():
    #SPARK_LOCAL_MASTER = "local[3]"
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).config('spark.driver.host', HOST_NAME).getOrCreate()
    default_conf = spark.sparkContext._conf#.getAll()
    print(f'Old Conf : {default_conf.getAll()}')
    conf = spark.sparkContext._conf.setAll([
        ('spark.executor.instances', 1)
        #, ('spark.driver.memory', '12g'), ('spark.executor.memory', '8g'), ('spark.driver.maxResultSize', '8g')
        , ('spark.driver.allowMultipleContexts', 'true'), ('spark.sql.shuffle.partitions', 8)
        ##,('spark.memory.offHeap.enabled', True), ('spark.memory.offHeap.size', '8g')
    ])
    spark.sparkContext.stop()
    
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).config(conf=default_conf).getOrCreate()
    new_conf = spark.sparkContext._conf
    print(f'Updated Conf : {new_conf.getAll()}')
    return spark

In [11]:
# 스파크 생성 
#spark = init_local_session()
# 스파크 클러스터를 실행해 두었다고 가정
# spark = SparkSession.builder.master("yarn").appName(SPARK_APP_NAME).getOrCreate()
# spark standard cluster 처럼 url 직접 지정하지 않고 protocol만 지정해서, yarn-site.xml 에 설정된 yarn RM 에 연결한다.  
spark = init_remote_session()
sc = spark.sparkContext
sc

RuntimeError: Java gateway process exited before sending its port number

In [14]:
# hive 에 testdb database 생성하고, intbl 테이블 생성 후, 1 row insert하고 조회.
# create table if not exists intbl ( eid int, ename string); insert into intbl values(1, 'a');
# 하이브 테이블이지만, 하이브로 접근하지 않았기 때문에 value : 1†a 형태로 읽힌다.  
spark.read.text("hdfs://namenode:9000/user/hive/warehouse/testdb.db/intbl").show()

+-----+
|value|
+-----+
|  1a|
+-----+



In [15]:
spark.stop()